In [1]:
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO 
from pyAudioAnalysis import MidTermFeatures as mF
import numpy as np 
import plotly.graph_objs as go 
import plotly
import IPython

import pandas as pd

import os, shutil
import time

import matplotlib.pyplot as plt

import random



/opt/anaconda3/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## OnlyCorrect

In [111]:
df_train = pd.read_csv('../Data/Mid_features/midFeaturesTrainFinal.csv')
df_test = pd.read_csv('../Data/Mid_features/midFeaturesTestFinal.csv')


In [112]:
df_train.sort_values(by=['FileID']);
df_test.sort_values(by=['FileID']);

In [113]:
summary = pd.read_csv('../Data/CREMADSummaryTable.csv')
summary

,Unnamed: 0,FileName,VoiceVote,VoiceLevel,FaceVote,FaceLevel,MultiModalVote,MultiModalLevel
0,1,1001_IEO_NEU_XX,N,69.1,N,92.22,N,64.78
1,2,1001_IEO_HAP_LO,N,71.67,H,57,H,57.38
2,3,1001_IEO_HAP_MD,N,67.71,H,62.62,H,56.56
3,4,1001_IEO_HAP_HI,H,63.5,H,68.25,H,73.2
4,5,1001_IEO_SAD_LO,N,73.71,N,73.5,N,74.8
...,...,...,...,...,...,...,...,...
7437,7438,1091_WSI_HAP_XX,N,68.12,H,65.12,H,64.8
7438,7439,1091_WSI_SAD_XX,N,55.67,N,71.57,N,48.8
7439,7440,1091_WSI_ANG_XX,A,34.4,A,50.43,A,67.11
7440,7441,1091_WSI_FEA_XX,S,39.67,F,68.29,F,51.71


In [114]:
summary.drop('Unnamed: 0', inplace = True, axis = 1)

### Example trial for Logical Comparison

In [29]:
summary['FileName'][0][9]==summary['VoiceVote'][0]

True

In [28]:
summary['VoiceVote'][0]

'N'

## Calculating the "correctness"

In [115]:
# Creating a new column and saving the real label code as a single letter

summary['filecode'] = np.nan
for i in np.arange(len(summary['FileName'])):
    summary['filecode'][i] = summary['FileName'][i][9];

<ipython-input-115-9c977deaf49b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary['filecode'][i] = summary['FileName'][i][9];


In [116]:
# Creating a new column and saving whether participants responded correctly to those in the "filecode"

summary['is_correct'] = np.nan
for i in np.arange(len(summary['FileName'])):
    x = (summary['filecode'][i]==summary['VoiceVote'][i])
    if x == True:
        summary['is_correct'][i] = 1
    elif x == False:
        summary['is_correct'][i] = 0;

<ipython-input-116-c1bf398b8e81>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary['is_correct'][i] = 1
<ipython-input-116-c1bf398b8e81>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary['is_correct'][i] = 0;


In [71]:
summary

,FileName,VoiceVote,VoiceLevel,FaceVote,FaceLevel,MultiModalVote,MultiModalLevel,is_correct,filecode
0,1001_IEO_NEU_XX,N,69.1,N,92.22,N,64.78,1.0,N
1,1001_IEO_HAP_LO,N,71.67,H,57,H,57.38,0.0,H
2,1001_IEO_HAP_MD,N,67.71,H,62.62,H,56.56,0.0,H
3,1001_IEO_HAP_HI,H,63.5,H,68.25,H,73.2,1.0,H
4,1001_IEO_SAD_LO,N,73.71,N,73.5,N,74.8,0.0,S
...,...,...,...,...,...,...,...,...,...
7437,1091_WSI_HAP_XX,N,68.12,H,65.12,H,64.8,0.0,H
7438,1091_WSI_SAD_XX,N,55.67,N,71.57,N,48.8,0.0,S
7439,1091_WSI_ANG_XX,A,34.4,A,50.43,A,67.11,1.0,A
7440,1091_WSI_FEA_XX,S,39.67,F,68.29,F,51.71,0.0,F


In [52]:
# Save them all into csv

summary.to_csv('../Data/Mid_features/summary_corrLabel.csv', index=False)

## Getting only train set values from the summary table

In [117]:
df_train.insert(loc = 4,
              column = 'is_correct',
              value = np.nan)

In [118]:
df_test.insert(loc = 4,
              column = 'is_correct',
              value = np.nan)


In [149]:
df_train.head()

,FileID,actorID,Emotion,SentenceID,is_correct,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1001_DFA_ANG_XX,1001,ANG,DFA,1.0,0.159956,0.012981,2.988679,0.250490,0.224054,...,0.024107,0.014803,0.017961,0.013412,0.008655,0.010352,0.009738,0.010600,0.004328,0.009167
1,1001_DFA_DIS_XX,1001,DIS,DFA,1.0,0.175069,0.006502,2.930843,0.258174,0.209151,...,0.022395,0.015510,0.008768,0.014533,0.009661,0.002533,0.004223,0.007513,0.003662,0.007296
2,1001_DFA_FEA_XX,1001,FEA,DFA,1.0,0.199849,0.016796,2.999322,0.272570,0.202443,...,0.007043,0.003129,0.006915,0.007791,0.013899,0.005247,0.003474,0.014306,0.005781,0.008250
3,1001_DFA_HAP_XX,1001,HAP,DFA,0.0,0.148663,0.007960,2.880264,0.235750,0.211387,...,0.021737,0.005675,0.009277,0.026797,0.010147,0.010658,0.017229,0.013203,0.010011,0.007488
4,1001_DFA_NEU_XX,1001,NEU,DFA,1.0,0.174283,0.010704,2.833565,0.256034,0.201942,...,0.031970,0.012929,0.017969,0.037496,0.013379,0.008354,0.005615,0.008907,0.007483,0.013592


In [150]:
for i in np.arange(len(df_train['FileID'])):    
    x = df_train['FileID'][i]
    y = summary.loc[summary['FileName'] == x]
    df_train['is_correct'][i] = y['is_correct']

<ipython-input-150-6a8e50c10df0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['is_correct'][i] = y['is_correct']


In [151]:
for i in np.arange(len(df_test['FileID'])):    
    x = df_test['FileID'][i]
    y = summary.loc[summary['FileName'] == x]
    df_test['is_correct'][i] = y['is_correct']

<ipython-input-151-69a25508e7eb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['is_correct'][i] = y['is_correct']


In [152]:
df_train.head()

,FileID,actorID,Emotion,SentenceID,is_correct,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1001_DFA_ANG_XX,1001,ANG,DFA,1.0,0.159956,0.012981,2.988679,0.250490,0.224054,...,0.024107,0.014803,0.017961,0.013412,0.008655,0.010352,0.009738,0.010600,0.004328,0.009167
1,1001_DFA_DIS_XX,1001,DIS,DFA,1.0,0.175069,0.006502,2.930843,0.258174,0.209151,...,0.022395,0.015510,0.008768,0.014533,0.009661,0.002533,0.004223,0.007513,0.003662,0.007296
2,1001_DFA_FEA_XX,1001,FEA,DFA,1.0,0.199849,0.016796,2.999322,0.272570,0.202443,...,0.007043,0.003129,0.006915,0.007791,0.013899,0.005247,0.003474,0.014306,0.005781,0.008250
3,1001_DFA_HAP_XX,1001,HAP,DFA,0.0,0.148663,0.007960,2.880264,0.235750,0.211387,...,0.021737,0.005675,0.009277,0.026797,0.010147,0.010658,0.017229,0.013203,0.010011,0.007488
4,1001_DFA_NEU_XX,1001,NEU,DFA,1.0,0.174283,0.010704,2.833565,0.256034,0.201942,...,0.031970,0.012929,0.017969,0.037496,0.013379,0.008354,0.005615,0.008907,0.007483,0.013592


In [153]:
df_test.head()

,FileID,actorID,Emotion,SentenceID,is_correct,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1006_DFA_ANG_XX,1006,ANG,DFA,1.0,0.205303,0.026108,2.936040,0.295857,0.220094,...,0.009290,0.011798,0.012141,0.005553,0.001191,0.005477,0.013535,0.017165,0.008230,0.008214
1,1006_DFA_DIS_XX,1006,DIS,DFA,1.0,0.142912,0.009082,2.815900,0.227427,0.198710,...,0.043360,0.034025,0.037411,0.016621,0.002165,0.006136,0.005255,0.008078,0.006201,0.013113
2,1006_DFA_FEA_XX,1006,FEA,DFA,0.0,0.150361,0.023924,2.758088,0.214044,0.187709,...,0.016754,0.011521,0.040531,0.011855,0.010751,0.012277,0.005750,0.014204,0.013292,0.012245
3,1006_DFA_HAP_XX,1006,HAP,DFA,1.0,0.117750,0.012498,2.945889,0.220303,0.219790,...,0.033142,0.021647,0.012535,0.028002,0.018135,0.012021,0.022196,0.020446,0.008418,0.012261
4,1006_DFA_NEU_XX,1006,NEU,DFA,1.0,0.118134,0.016632,2.802065,0.193803,0.183517,...,0.026941,0.010169,0.038335,0.015219,0.011879,0.015302,0.008216,0.013460,0.016431,0.012039


In [154]:
df_train.to_csv('../Data/Mid_features/df_train_corrLabel.csv', index=False)

In [155]:
df_test.to_csv('../Data/Mid_features/df_test_all.csv', index=False)

In [156]:
df_test_corrlabel = df_test[df_test['is_correct'] == 1]

In [157]:
df_test_corrlabel

,FileID,actorID,Emotion,SentenceID,is_correct,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
0,1006_DFA_ANG_XX,1006,ANG,DFA,1.0,0.205303,0.026108,2.936040,0.295857,0.220094,...,0.009290,0.011798,0.012141,0.005553,0.001191,0.005477,0.013535,0.017165,0.008230,0.008214
1,1006_DFA_DIS_XX,1006,DIS,DFA,1.0,0.142912,0.009082,2.815900,0.227427,0.198710,...,0.043360,0.034025,0.037411,0.016621,0.002165,0.006136,0.005255,0.008078,0.006201,0.013113
3,1006_DFA_HAP_XX,1006,HAP,DFA,1.0,0.117750,0.012498,2.945889,0.220303,0.219790,...,0.033142,0.021647,0.012535,0.028002,0.018135,0.012021,0.022196,0.020446,0.008418,0.012261
4,1006_DFA_NEU_XX,1006,NEU,DFA,1.0,0.118134,0.016632,2.802065,0.193803,0.183517,...,0.026941,0.010169,0.038335,0.015219,0.011879,0.015302,0.008216,0.013460,0.016431,0.012039
5,1006_DFA_SAD_XX,1006,SAD,DFA,1.0,0.115795,0.014708,2.844136,0.204687,0.210093,...,0.022490,0.013506,0.030509,0.026745,0.006870,0.014035,0.009864,0.018840,0.012973,0.013929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,1089_TSI_NEU_XX,1089,NEU,TSI,1.0,0.301379,0.019511,2.851780,0.379223,0.250684,...,0.028692,0.024063,0.006848,0.013698,0.002913,0.013846,0.009565,0.003859,0.003258,0.009791
1194,1089_WSI_ANG_XX,1089,ANG,WSI,1.0,0.229983,0.045743,2.994412,0.316311,0.226412,...,0.002990,0.001033,0.004155,0.003678,0.001318,0.019570,0.003408,0.002347,0.026694,0.009494
1195,1089_WSI_DIS_XX,1089,DIS,WSI,1.0,0.383122,0.012807,2.857422,0.432665,0.261348,...,0.004918,0.013561,0.010522,0.009827,0.005967,0.010294,0.010915,0.004520,0.010917,0.005369
1198,1089_WSI_NEU_XX,1089,NEU,WSI,1.0,0.139094,0.008446,2.883838,0.221700,0.209398,...,0.023018,0.010559,0.011100,0.021006,0.007919,0.003927,0.009988,0.009861,0.020112,0.007493


In [158]:
df_test_corrlabel.to_csv('../Data/Mid_features/df_test_only_corrLabel.csv', index=False)

## Do the same for Categories.csv files

In [137]:
df_cat_train = pd.read_csv('../Data/Categories_train.csv')
df_cat_test = pd.read_csv('../Data/Categories_test.csv')
df_cat = pd.read_csv('../Data/Categories.csv')

In [138]:
df_cat_train.sort_values(by=['FileID']);
df_cat_test.sort_values(by=['FileID']);
df_cat.sort_values(by=['FileID']);

In [139]:
df_cat

,FileID,ActorID,SentenceID,Emotion,Intensity
0,1001_DFA_ANG_XX,1001,DFA,ANG,XX
1,1001_DFA_DIS_XX,1001,DFA,DIS,XX
2,1001_DFA_FEA_XX,1001,DFA,FEA,XX
3,1001_DFA_HAP_XX,1001,DFA,HAP,XX
4,1001_DFA_NEU_XX,1001,DFA,NEU,XX
...,...,...,...,...,...
6071,1091_WSI_DIS_XX,1091,WSI,DIS,XX
6072,1091_WSI_FEA_XX,1091,WSI,FEA,XX
6073,1091_WSI_HAP_XX,1091,WSI,HAP,XX
6074,1091_WSI_NEU_XX,1091,WSI,NEU,XX


In [143]:
df_cat_train.insert(loc = 5,
              column = 'is_correct',
              value = np.nan)
df_cat_test.insert(loc = 5,
              column = 'is_correct',
              value = np.nan)


In [136]:
df_cat

,FileID,ActorID,SentenceID,Emotion,Intensity,is_correct
0,1001_DFA_ANG_XX,1001,DFA,ANG,XX,NaN
1,1001_DFA_DIS_XX,1001,DFA,DIS,XX,NaN
2,1001_DFA_FEA_XX,1001,DFA,FEA,XX,NaN
3,1001_DFA_HAP_XX,1001,DFA,HAP,XX,NaN
4,1001_DFA_NEU_XX,1001,DFA,NEU,XX,NaN
...,...,...,...,...,...,...
6071,1091_WSI_DIS_XX,1091,WSI,DIS,XX,NaN
6072,1091_WSI_FEA_XX,1091,WSI,FEA,XX,NaN
6073,1091_WSI_HAP_XX,1091,WSI,HAP,XX,NaN
6074,1091_WSI_NEU_XX,1091,WSI,NEU,XX,NaN


In [144]:
for i in np.arange(len(df_cat_train['FileID'])):    
    x = df_cat_train['FileID'][i]
    y = summary.loc[summary['FileName'] == x]
    df_cat_train['is_correct'][i] = y['is_correct']

<ipython-input-144-8e45805fefa4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat_train['is_correct'][i] = y['is_correct']


In [145]:
for i in np.arange(len(df_cat_test['FileID'])):    
    x = df_cat_test['FileID'][i]
    y = summary.loc[summary['FileName'] == x]
    df_cat_test['is_correct'][i] = y['is_correct']

<ipython-input-145-060a48e6f06d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat_test['is_correct'][i] = y['is_correct']


In [140]:
df_cat.insert(loc = 5,
              column = 'is_correct',
              value = summary['is_correct'])

In [146]:
df_cat_train

,FileID,ActorID,SentenceID,Emotion,Intensity,is_correct,Old_Dur,New_Dur
0,1001_DFA_ANG_XX,1001,DFA,ANG,XX,1.0,2.275562,1.34
1,1001_DFA_DIS_XX,1001,DFA,DIS,XX,1.0,2.335688,1.46
2,1001_DFA_FEA_XX,1001,DFA,FEA,XX,1.0,2.168813,1.06
3,1001_DFA_HAP_XX,1001,DFA,HAP,XX,0.0,1.868500,1.22
4,1001_DFA_NEU_XX,1001,DFA,NEU,XX,1.0,2.035375,1.24
...,...,...,...,...,...,...,...,...
4871,1091_WSI_DIS_XX,1091,WSI,DIS,XX,0.0,2.502500,1.22
4872,1091_WSI_FEA_XX,1091,WSI,FEA,XX,0.0,2.502500,1.14
4873,1091_WSI_HAP_XX,1091,WSI,HAP,XX,0.0,2.168813,1.50
4874,1091_WSI_NEU_XX,1091,WSI,NEU,XX,1.0,2.335688,1.20


In [147]:
df_cat_test

,FileID,ActorID,SentenceID,Emotion,Intensity,is_correct,Old_Dur,New_Dur
0,1006_DFA_ANG_XX,1006,DFA,ANG,XX,1.0,2.602625,1.46
1,1006_DFA_DIS_XX,1006,DFA,DIS,XX,1.0,2.769437,1.62
2,1006_DFA_FEA_XX,1006,DFA,FEA,XX,0.0,2.369000,1.22
3,1006_DFA_HAP_XX,1006,DFA,HAP,XX,1.0,2.302313,1.46
4,1006_DFA_NEU_XX,1006,DFA,NEU,XX,1.0,2.569250,1.40
...,...,...,...,...,...,...,...,...
1195,1089_WSI_DIS_XX,1089,WSI,DIS,XX,1.0,4.070750,1.60
1196,1089_WSI_FEA_XX,1089,WSI,FEA,XX,0.0,2.302313,1.32
1197,1089_WSI_HAP_XX,1089,WSI,HAP,XX,0.0,2.202187,1.76
1198,1089_WSI_NEU_XX,1089,WSI,NEU,XX,1.0,2.569250,1.06


In [148]:
df_cat_train.to_csv('../Data/Categories_train_corr.csv', index=False)
df_cat_test.to_csv('../Data/Categories_test_corr.csv', index=False)
df_cat.to_csv('../Data/Categories_corr.csv', index=False)